# Launch Sites Locations Analysis with Folium
___

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis labs, we have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, we will be performing more interactive visual analytics using `Folium`.

## Objectives
We will perform following tasks:

*   Mark all launch sites on a map
*   Mark the success/failed launches for each site on the map
*   Calculate the distances between a launch site to its proximities

After completed the above tasks, we should be able to find some geographical patterns about launch sites.

In [1]:
!pip3 install folium==0.12.1
!pip3 install wget

  Attempting uninstall: folium
    Found existing installation: folium 0.12.1.post1
    Uninstalling folium-0.12.1.post1:
      Successfully uninstalled folium-0.12.1.post1


In [2]:
import folium
import wget
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

### Mark all launch sites on a map
First, let's try to add each site's location on a map using site's latitude and longitude coordinates


In [6]:
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)
spacex_df.head(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
5,7,2013-12-03,22:41:00,F9 v1.1,CCAFS LC-40,SES-8,3170.0,GTO,SES,No attempt,0,28.562302,-80.577356
6,8,2014-01-06,22:06:00,F9 v1.1,CCAFS LC-40,Thaicom 6,3325.0,GTO,Thaicom,No attempt,0,28.562302,-80.577356
7,9,2014-04-18,19:25:00,F9 v1.1,CCAFS LC-40,SpaceX CRS-3,2296.0,LEO (ISS),NASA (CRS),Controlled (ocean),0,28.562302,-80.577356
8,10,2014-07-14,15:15:00,F9 v1.1,CCAFS LC-40,OG2 Mission 1 6 Orbcomm-OG2 satellites,1316.0,LEO,Orbcomm,Controlled (ocean),0,28.562302,-80.577356
9,11,2014-08-05,8:00:00,F9 v1.1,CCAFS LC-40,AsiaSat 8,4535.0,GTO,AsiaSat,No attempt,0,28.562302,-80.577356


In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


The coordinates aboove are just plain numbers that cannot give us intuitive geographic insights. Let's visualize those locations by pinning them on a map.

First let's crete a `Folium Map` object, with an initial location to be NASAS Johnson Space Center at Houston, Texas.

In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Now, let's add a circle for each launch site in data frame `launch_sites`

In [10]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for lat, long, label in zip(launch_sites_df.Lat, launch_sites_df.Long, launch_sites_df['Launch Site']):
    circle = folium.Circle([lat, long], color='#d35400', radius=100, fill=True
                          ).add_child(folium.Popup(label))
    marker = folium.Marker([lat, long], icon=DivIcon(icon_size=(20,20),
                                                     icon_anchor = (0, 0),
                                                     html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % label
                                                    ))
    site_map.add_child(marker)
    site_map.add_child(circle)

site_map

We can see that each launchsite are close to the coast, and all launch sites are quite close to equator line

### Mark the success/failed launches for each site on the map
Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`

Let's first create a `MarkerCluster` object

In [12]:
marker_cluster = MarkerCluster()

In [13]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
def mark_color(x):
    if x == 1:
        return 'green'
    else:
        return 'red'
    
launch_sites_df['marker_color'] = spacex_df['class'].apply(mark_color)
launch_sites_df.head(15)

,Launch Site,Lat,Long,marker_color
0,CCAFS LC-40,28.562302,-80.577356,red
1,CCAFS SLC-40,28.563197,-80.576820,red
2,KSC LC-39A,28.573255,-80.646895,red
3,VAFB SLC-4E,34.632834,-120.610745,red


In [14]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [15]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker = folium.Marker([record['Lat'], record['Long']], icon = folium.Icon(color='white', icon_color = record['marker_color']))
    marker_cluster.add_child(marker)


site_map

###  Calculate the distances between a launch site to its proximities
Next, we need to explore and analyze the proximities of launch sites.
Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [16]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:

In [17]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [18]:
# find coordinate of the closet mountain
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distace_promontorymt = calculate_distance(34.63283,-120.61074,34.60412,-120.64181)
distace_promontorymt

4.276271169428826

In [19]:
distance_marker = folium.Marker(
    [34.60412,-120.64181],
    icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distace_promontorymt),
       )
    )
site_map.add_child(distance_marker)

In [20]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
lines=folium.PolyLine(locations = [[34.63283,-120.61074],[34.60412,-120.64181]], weight = 1)
site_map.add_child(lines)

In [21]:
names = ['KSC LC-39A','CCAFS LC-40','CCAFS SLC-40', 'VAFB SLC-4E']
ls_coor = [[28.57323, -80.64692], [28.56231, -80.57737] ,[28.56321, -80.57684] ,[34.63289, -120.6107]]
railways_coor = [[28.57298, -80.65392],[28.5721,-80.58528],[28.5721,-80.58528],[34.63338, -120.62485]]
highways_coor = [[28.52598, -80.65664],[28.52598, -80.65664],[28.52598, -80.65664],[34.67889, -120.57701]]
coastline_coor = [[28.58022, -80.64226],[28.56347, -80.568],[28.56347, -80.568],[34.63308, -120.6263]]
cities_coor = [[28.61105, -80.82367],[28.39623, -80.6012],[28.39623, -80.6012],[34.63829, -120.45822]]
distance_rw = []
distance_hw = []
distance_cl = []
distance_ct= []

In [22]:
for i in range (0, len(names)): 
    distance_rw.append(calculate_distance(ls_coor[i][0], ls_coor[i][1], railways_coor[i][0], railways_coor[i][1]))
    distance_hw.append(calculate_distance(ls_coor[i][0], ls_coor[i][1], highways_coor[i][0], highways_coor[i][1]))
    distance_cl.append(calculate_distance(ls_coor[i][0], ls_coor[i][1], coastline_coor[i][0], coastline_coor[i][1]))
    distance_ct.append(calculate_distance(ls_coor[i][0], ls_coor[i][1], cities_coor[i][0], cities_coor[i][1]))

In [24]:
print(distance_rw)
print(distance_hw)
print(distance_cl)
print(distance_ct)
dict = {'rw': distance_rw, 'hw': distance_hw, 'cl': distance_cl, 'ct': distance_ct} 
df_f = pd.DataFrame(dict)
df_f = df_f.T
df_f = df_f.rename(columns={0:names[0], 1:names[1], 2:names[2], 3:names[3]})
df_f = df_f.reset_index()
df_f['index'] = df_f['index'].replace('rw','Nearest Railway').replace('hw','Nearest Highway').replace('cl','Nearest Coastline').replace('ct','Nearest City')
df_f = df_f.set_index('index')
df_f

df_f.T.describe()



[0.6843453711132712, 1.3352448575255096, 1.2874659918356248, 1.296165417417071]
[5.340724929138304, 8.736220494978525, 8.82864690445542, 5.973361651132563]
[0.9009415480567923, 0.9244261845244535, 0.8640809104463676, 1.4278831323024965]
[17.767504246772905, 18.61939378374256, 18.72523691672543, 13.967594469611509]


,KSC LC-39A,CCAFS LC-40,CCAFS SLC-40,VAFB SLC-4E
index,,,,
Nearest Railway,0.684345,1.335245,1.287466,1.296165
Nearest Highway,5.340725,8.736220,8.828647,5.973362
Nearest Coastline,0.900942,0.924426,0.864081,1.427883
Nearest City,17.767504,18.619394,18.725237,13.967594


In [25]:
df_f.T.describe()

index,Nearest Railway,Nearest Highway,Nearest Coastline,Nearest City
count,4.000000,4.000000,4.000000,4.000000
mean,1.150805,7.219738,1.029333,17.269932
std,0.311667,1.823225,0.266858,2.242913
min,0.684345,5.340725,0.864081,13.967594
25%,1.136686,5.815202,0.891726,16.817527
50%,1.291816,7.354791,0.912684,18.193449
75%,1.305935,8.759327,1.050290,18.645855
max,1.335245,8.828647,1.427883,18.725237


# Change Log
| Date (YYYY-MM-DD) | Version | Changed By | Change Description                  |
| ----------------- | ------- | ---------- | ----------------------------------- |
| 2022-04-12        | 1.0     | Yoel       | Created                             |

The code are learned from <b>IBM Data Science Professional Certification</b> course on <b>Coursera</b>